### 크롤링을 통한 중고차 데이터 셋 확보
https://github.com/sobin98/used_car_prediction/blob/main/webcrawling_usedcar.py#L1
- https://velog.io/@sobing/usedCarPrediction

In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import json

In [20]:
# 현대 : 그랜저, 스타랙스, 포터, 아반떼, 쏘나타, 싼타페.| 제네시스 : G80, G90, | 기아 : 카니발, 봉고, k7, 레이, 모닝 |
# 쉐보레/대우 | 삼성 (르노) | 수입: 벤츠 : E클래스, s클, 마이바흐 s클, g바겐, 포르쉐 : 911, 카이엔, 파나메라, |

totalurl = [("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=49&group_no=5&page=1&order=S11&view_size=20", 7),
       ("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=49&group_no=17&page=1&order=S11&view_size=20", 5),
       ("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=49&group_no=42&page=1&order=S11&view_size=20", 5),
       ("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=49&group_no=22&page=1&order=S11&view_size=20", 4),
       ("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=49&group_no=21&page=1&order=S11&view_size=20", 3),
       ("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=49&group_no=20&page=1&order=S11&view_size=20", 2),
       ("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=1010&group_no=893&page=1&order=S11&view_size=20", 4),
       ("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=1010&group_no=893&page=1&order=S11&view_size=20", 4),
       ("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=3&group_no=76&page=1&order=S11&view_size=20", 9),
       ("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=3&group_no=59&page=1&order=S11&view_size=20", 4),
       ("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=3&group_no=45&page=1&order=S11&view_size=20", 3),
       ("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=3&group_no=50&page=1&order=S11&view_size=20", 2),
       ("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=3&group_no=55&page=1&order=S11&view_size=20", 2),
       ("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=8&page=1&order=S11&view_size=20", 7),
       ("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=K&maker_no=26&page=1&order=S11&view_size=20", 5),
       ("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=I&maker_no=21&group_no=356&page=1&order=S11&view_size=20", 6),
       ("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=I&maker_no=21&group_no=358&page=1&order=S11&view_size=20", 16),
       ("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=I&maker_no=21&group_no=359&page=1&order=S11&view_size=20", 5),
       ("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=I&maker_no=21&group_no=362&page=1&order=S11&view_size=20", 8),
       ("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=I&maker_no=43&group_no=250&page=1&order=S11&view_size=20", 10),
       ("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=I&maker_no=43&group_no=251&page=1&order=S11&view_size=20", 7),
       ("https://www.bobaedream.co.kr/mycar/mycar_list.php?gubun=I&maker_no=43&group_no=252&page=1&order=S11&view_size=20", 6)
      ]
df_cars=[]

urls=[]
for i in totalurl:
    pagenum=i[1]
    for j in range(pagenum):
        url=i[0].format(str(j))
        urls.append(url)
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}


In [23]:
info=["이름","차량번호","링크","연식","주행거리","연료","배기량","색상","보증정보","가격","신차대비가격"]

coloptions=["옵션_선루프","옵션_파노라마선루프","옵션_열선앞","옵션_열선뒤","옵션_전방센서"
     ,"옵션_후방센서","옵션_전방캠","옵션_후방캠","옵션_어라운드뷰","옵션_네비순정"]

findoptions=["선루프","파노라마선루프","열선시트(앞좌석)","열선시트(뒷좌석)","전방센서"
     ,"후방센서","전방카메라","후방카메라","어라운드뷰","네비게이션(순정)"]
colacci_info1=["보험이력등록","소유자변경횟수","사고상세_전손","사고상세_침수전손","사고상세_침수분손","사고상세_도난","보험_내차피해(횟수)","보험_내차피해(가격)","사고상세_타차가해(횟수)","보험_타차피해(가격)"]

cols=info+coloptions+colacci_info1 #분석과정에서 아웃라이어의 판별여부를 위해 링크추가

#옵션이름 받아서 확인여부하는 함수
def option_check(soupobject,option_name):
    check = soupobject.find("button", text=option_name).find_parent().find_previous_sibling().get_attribute_list('checked')

    if check[0]=='':
        return '유'
    else:
        return '무'
        
for url in urls:

    res=requests.get(url, headers = headers)
    res.raise_for_status()
    requests.adapters.DEFAULT_RETRIES = 10
    soup=BeautifulSoup(res.text,"lxml")

    cars=soup.find_all("li",attrs={"class":"product-item"})
    links=[]
    #한 url마다 들어있는 모든 차들에 대해 실행
    for car in cars:
        link = "https://www.bobaedream.co.kr" + car.a["href"]
        links.append(link)
    for link in links:
        print(link)


        res2=requests.get(link,timeout=5, headers = headers)
        res2.raise_for_status()
        soup2 = BeautifulSoup(res2.text, "lxml")
        infobox = soup2.find("div", attrs={"class": "info-util box"})
        try:
            ratiopr = infobox.find("b")
        except:
            continue

        name=soup2.find("h3",attrs={"class":"tit"})
        state=soup2.find("div",attrs={"class":"tbl-01 st-low"})
        galdata=soup2.find("div",attrs={"class":"gallery-data"})
        carnumber=galdata.find("b")

        year=state.find("th",text='연식').find_next_sibling("td")
        km=state.find("th",text='주행거리').find_next_sibling("td")
        fuel=state.find("th",text='연료').find_next_sibling("td")
        amount=state.find("th",text='배기량').find_next_sibling("td")
        color=state.find("th",text='색상').find_next_sibling("td")
        guarn=state.find("b",text='보증정보').find_next("td")
        price=soup2.find("span",attrs={"class":"price"})

        option_table=soup2.find("div",attrs={"class":"tbl-option"})
        checkoptions=[]
        if option_table.find("th",text='외관')!=None:
            for option in findoptions:
                checkoptions.append(option_check(option_table,option))
        else:
            checkoptions=['']*len(coloptions)

        if infobox.find("span",attrs={"class":"round-ln insurance"}).find_next("i").find_next("em")==None:
            acc1 = '미등록'
        else:
            acc1 = '등록'

        findacci_info1=[]
        try:
            if acc1=='등록':
                acc1table=soup2.find("div",attrs={"class":"info-insurance"})
                try:
                    insurdt1 = acc1table.find("th", text="Vehicle number/owner change").find_next_sibling("td").get_text()[-2]
                except AttributeError:
                    insurdt1 = ''

                try:
                    insuraccis1 = acc1table.find("th", text="auto insurance special accident").find_next_sibling("td").get_text().split('/')
                    insurdt2 = insuraccis1[0][-2]
                    insurdt3 = insuraccis1[1][-2]
                    insurdt4 = insuraccis1[2][-2]
                    insurdt5 = insuraccis1[3][-1]
                except (AttributeError, IndexError):
                    insurdt2 = ''
                    insurdt3 = ''
                    insurdt4 = ''
                    insurdt5 = ''

                try:
                    insuraccis2 = acc1table.find("th", text="Insurance accident (insurance damage)").find_next_sibling("td").get_text().split('time')
                    insurdt6 = insuraccis2[0]
                    insurdt7 = insuraccis2[1][2:-2]
                except (AttributeError, IndexError):
                    insurdt6 = ''
                    insurdt7 = ''

                try:
                    insuraccis3 = acc1table.find("th", text="Insurance accident (other damage)").find_next_sibling("td").get_text().split('time')
                    insurdt8 = insuraccis3[0]
                    insurdt9 = insuraccis3[1][2:-2]
                except (AttributeError, IndexError):
                    insurdt8 = ''
                    insurdt9 = ''

                findacci_info1 = [insurdt1, insurdt2, insurdt3, insurdt4, insurdt5, insurdt6, insurdt7, insurdt8, insurdt9]

            else:
                findacci_info1=['']*(len(colacci_info1)-1)
        except IndexError:
                findacci_info1 = ['']*(len(colacci_info1)-1)

        temp=[name.get_text(),carnumber.get_text(),link,year.get_text(),km.get_text(),fuel.get_text(),amount.get_text(),
          color.get_text(),guarn.get_text(),price.get_text(),ratiopr.get_text()]+checkoptions+[acc1]+findacci_info1
        df_cars.append(temp)
        
    df_cars=pd.DataFrame(data=df_cars,columns=cols)

https://www.bobaedream.co.kr/mycar/mycar_view.php?no=2204876&gubun=K


C:\Users\user\AppData\Local\Temp\ipykernel_6252\2297991287.py:52: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  year=state.find("th",text='연식').find_next_sibling("td")
C:\Users\user\AppData\Local\Temp\ipykernel_6252\2297991287.py:53: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  km=state.find("th",text='주행거리').find_next_sibling("td")
C:\Users\user\AppData\Local\Temp\ipykernel_6252\2297991287.py:54: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  fuel=state.find("th",text='연료').find_next_sibling("td")
C:\Users\user\AppData\Local\Temp\ipykernel_6252\2297991287.py:55: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  amount=state.find("th",text='배기량').find_next_sibling("td")
C:\Users\user\AppData\Local\Temp\ipykernel_6252\2297991287.py:56: DeprecationWarning: The 'text' argu

https://www.bobaedream.co.kr/mycar/mycar_view.php?no=2204251&gubun=K
https://www.bobaedream.co.kr/mycar/mycar_view.php?no=2203530&gubun=K
https://www.bobaedream.co.kr/mycar/mycar_view.php?no=2196777&gubun=K
https://www.bobaedream.co.kr/mycar/mycar_view.php?no=2195463&gubun=K
https://www.bobaedream.co.kr/mycar/mycar_view.php?no=2204170&gubun=K
https://www.bobaedream.co.kr/mycar/mycar_view.php?no=2199981&gubun=K
https://www.bobaedream.co.kr/mycar/mycar_view.php?no=2204308&gubun=K
https://www.bobaedream.co.kr/mycar/mycar_view.php?no=2204309&gubun=K
https://www.bobaedream.co.kr/mycar/mycar_view.php?no=2199508&gubun=K
https://www.bobaedream.co.kr/mycar/mycar_view.php?no=2205153&gubun=K
https://www.bobaedream.co.kr/mycar/mycar_view.php?no=2205149&gubun=K
https://www.bobaedream.co.kr/mycar/mycar_view.php?no=2205146&gubun=K
https://www.bobaedream.co.kr/mycar/mycar_view.php?no=2202297&gubun=K
https://www.bobaedream.co.kr/mycar/mycar_view.php?no=2199060&gubun=K
https://www.bobaedream.co.kr/mycar

AttributeError: 'DataFrame' object has no attribute 'append'